In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from scipy import signal
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from glob import glob
import math
from sklearn.model_selection import train_test_split
from PIL.ImageOps import invert
from keras.utils import plot_model
from sklearn.metrics import classification_report
from gprGAN import gprGAN
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from keras.datasets import mnist
from keras.layers.merge import _Merge, multiply
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from functools import partial
import tensorflow as tf
from keras.utils import to_categorical

np.random.seed(1234)
tf.set_random_seed(1234)

os.environ["CUDA_VISIBLE_DEVICES"]="0"


from keras.backend.tensorflow_backend import set_session 
config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True 
# dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran) 
# (nothing gets printed in Jupyter, only if you run it standalone) 
sess = tf.Session(config=config)
set_session(sess) # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [2]:
concrete_paths = glob('../test-set/preprocessed/*_concrete_*')
metallic_paths = glob('../test-set/preprocessed/*_pec_*')
plastic_paths = glob('../test-set/preprocessed/*_pvc_*')

def load_img(path):
    img = image.load_img(path, color_mode='grayscale', target_size=(256,256), interpolation='hamming')
    img = invert(img)
    return image.img_to_array(img)

concrete_scans = np.array([load_img(path) for path in concrete_paths])
metallic_scans = np.array([load_img(path) for path in metallic_paths])
plastic_scans = np.array([load_img(path) for path in plastic_paths])

concrete_labels = np.zeros((len(concrete_scans),))
metallic_labels = np.ones((len(metallic_scans),))
plastic_labels = np.ones((len(plastic_scans),)) * 2

y = np.concatenate([concrete_labels, metallic_labels, plastic_labels]).astype(np.int32)
a_scans = np.concatenate([concrete_scans, metallic_scans, plastic_scans])

a_scans = (a_scans.astype(np.float32) - 127.5) / 127.5
#y = to_categorical(y, 3)

X_train, X_test, y_train, y_test = train_test_split(a_scans, y, test_size=0.3)
print(X_train.shape, y_train.shape)

(117, 256, 256, 1) (117,)


In [3]:
def build_classifier():
    model = Sequential()
    model.add(Conv2D(16, kernel_size=5, strides=2, padding="same", input_shape=(256, 256, 1)))
    model.add(LeakyReLU(alpha=0.3))
    #model.add(BatchNormalization(momentum=0.8))
    #model.add(Dropout(0.75))
    model.add(Conv2D(32, kernel_size=5, strides=2, padding="same"))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.3))
    #model.add(Dropout(0.75))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    img = Input(shape=(256, 256, 1))
        
    label = model(img)
        
    return Model(img, label)

callback = keras.callbacks.ModelCheckpoint('./checkpoints/aux.h5', monitor='val_acc', verbose=0, save_best_only=True)

In [4]:
model = build_classifier()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=2, epochs=1000, validation_data=[X_test, y_test], callbacks=[callback], verbose=0)

In [5]:
model.load_weights('./checkpoints/aux.h5')
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8235294117647058
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       0.69      0.82      0.75        11
           2       0.75      0.60      0.67        15

   micro avg       0.82      0.82      0.82        51
   macro avg       0.79      0.79      0.79        51
weighted avg       0.82      0.82      0.82        51

